# SQL and Big Query

# Lib Imports

In [29]:
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

## Declare Table and Establish Connection

In [30]:
table_name = '../data/recipes_table.db'

In [31]:
table_connection = sq3.connect(table_name)

## Lets Start by Creating our Recipe Table

We're going to be creating three tables. The recipes table, the ingredients table and cooking instructions table. The ingredients and cooking instructions are going to use the foreign key,

In [32]:
create_ingredients_table_query = """
    CREATE TABLE ingredients(
        recipe_id VARCHAR(36) NOT NULL PRIMARY KEY,
        ingredient_string VARCHAR(12) NOT NULL,
        quantity VARCHAR(10) NOT NULL,
        measurement VARCHAR(5) NOT NULL
    )
"""

create_cooking_steps_table_query = """
    CREATE TABLE cooking_steps(
        recipe_id VARCHAR(36) NOT NULL PRIMARY KEY,
        cooking_step_num INT NOT NULL,
        cooking_step_string VARCHAR(255) NOT NULL
    )
"""

create_recipes_table_query = """
    CREATE TABLE recipes(
        recipe_id VARCHAR(36) NOT NULL PRIMARY KEY,
        name VARCHAR(12) NOT NULL,
        description VARCHAR(250) NOT NULL
    )
"""

In [33]:
table_connection.execute(create_ingredients_table_query)
table_connection.execute(create_cooking_steps_table_query)

In [34]:
table_connection.execute(create_recipes_table_query)

Now lets check that all of the columns are in place.

In [37]:
get_all_recipes_query = """
SELECT recipes.*, cooking_steps.cooking_step_num, cooking_steps.cooking_step_string
FROM recipes
INNER JOIN cooking_steps ON cooking_steps.recipe_id = recipes.recipe_id
"""

In [38]:
recipes_generator_objects = pd.read_sql(get_all_recipes_query, table_connection)
recipes_generator_objects.head()

DatabaseError: Execution failed on sql '
SELECT recipes.*, cooking_steps.cooking_step_num, cooking_steps.cooking_step_string
FROM recipes
INNER JOIN cooking_steps ON cooking_steps.recipe_id = recipes.recipe_id
': no such column: recipes.recipe_id

In [33]:
ids = [1, 2, 3]
names = ['chicken curry', 'pasta', 'tuna sandwich']
descriptions = ['this is a curry', 'this is a pasta', 'this is tuna']

In [35]:
sql_tuples = [(ids[i], names[i], descriptions[i]) for i in range(len(ids))]

In [38]:
insert_recipes_query = "INSERT INTO Recipes(id, name, description) VALUES(?, ?, ?)"

In [39]:
table_connection.executemany(insert_recipes_query, sql_tuples)
table_connection.commit()

In [41]:
recipes = pd.read_sql(select_all, table_connection)
recipes.head()

,id,name,description
0,1,chicken curry,this is a curry
1,2,pasta,this is a pasta
2,3,tuna sandwich,this is tuna


## Add some Ingredients

Lets modify the recipe table to add ingredient ids.

# Executing Some Queries

In [24]:
recipes_generator = pd.read_sql(select_all,
                                table_connection)
next(iter(recipes_generator))

'id'

### Reading data

Now that we've got a connection to our database, we can perform queries, and load their results in as Pandas DataFrames


In [7]:
query = '''
SELECT * FROM rock_songs;
'''

observations = pds.read_sql(query, con)
observations.head()

,Song,Artist,Release_Year,PlayCount
0,Caught Up in You,.38 Special,1982.0,82
1,Hold On Loosely,.38 Special,1981.0,85
2,Rockin' Into the Night,.38 Special,1980.0,18
3,Art For Arts Sake,10cc,1975.0,1
4,Kryptonite,3 Doors Down,2000.0,13


## Common parameters

There are a number of common paramters that can be used to read in SQL data with formatting:

 - coerce_float: Attempt to force numbers into floats
 - parse_dates: List of columns to parse as dates
 - chunksize: Number of rows to include in each chunk
 
Let's have a look at using some of these parameters


In [12]:
query='''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

In [21]:
observations_generator = pd.read_sql(query, 
                               con,
                               coerce_float=True,
                               parse_dates=['Release_Year'],
                               chunksize=5)

In [22]:
for i, obs in enumerate(observations_generator):
    if i < 5:
        print(f'Observations index: {i}'.format(i))
        display(obs)

Observations index: 0


,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1970-01-01 00:32:47,23,6.565217
1,Led Zeppelin,1970-01-01 00:32:49,18,21.000000
2,The Beatles,1970-01-01 00:32:45,15,3.800000
3,The Beatles,1970-01-01 00:32:48,13,13.000000
4,The Beatles,1970-01-01 00:32:49,13,15.000000


Observations index: 1


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:50,12,13.166667
1,Led Zeppelin,1970-01-01 00:32:55,12,14.166667
2,Pink Floyd,1970-01-01 00:32:59,11,41.454545
3,Pink Floyd,1970-01-01 00:32:53,10,29.100000
4,The Doors,1970-01-01 00:32:47,10,28.900000


Observations index: 2


,Artist,Release_Year,num_songs,avg_plays
0,Fleetwood Mac,1970-01-01 00:32:57,9,35.666667
1,Jimi Hendrix,1970-01-01 00:32:47,9,24.888889
2,The Beatles,1970-01-01 00:32:43,9,2.444444
3,The Beatles,1970-01-01 00:32:44,9,3.111111
4,Elton John,1970-01-01 00:32:53,8,18.500000


Observations index: 3


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:51,8,47.750000
1,Led Zeppelin,1970-01-01 00:32:53,8,34.125000
2,Boston,1970-01-01 00:32:56,7,69.285714
3,Rolling Stones,1970-01-01 00:32:49,7,36.142857
4,Van Halen,1970-01-01 00:32:58,7,51.142857


Observations index: 4


,Artist,Release_Year,num_songs,avg_plays
0,Bruce Springsteen,1970-01-01 00:32:55,6,7.666667
1,Bruce Springsteen,1970-01-01 00:33:04,6,11.500000
2,Creedence Clearwater Revival,1970-01-01 00:32:49,6,23.833333
3,Creedence Clearwater Revival,1970-01-01 00:32:50,6,18.833333
4,Def Leppard,1970-01-01 00:33:07,6,32.000000
